In [2]:
import sys
import platform
import pandas as pd 
import numpy as np 

print("Python version:", sys.version)
print("Executable:", sys.executable)
print("Platform:", platform.platform())


Python version: 3.8.0 | packaged by conda-forge | (default, Nov 22 2019, 19:04:36) [MSC v.1916 64 bit (AMD64)]
Executable: c:\development_projects\FilmFlow\venv\python.exe
Platform: Windows-10-10.0.22621-SP0


**Read data**

In [3]:
df1 = pd.read_csv("../data/movie_metadata.csv")
df2 = pd.read_csv("../data/movies_metadata.csv")
df3 = pd.read_csv("../data/credits.csv")


C:\Users\prasa\AppData\Local\Temp\ipykernel_20948\4206685961.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../data/movies_metadata.csv")


**Read Wiki data 2018-2023**

In [4]:
link18 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
link19 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
link20 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
link21 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"
link22 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
link23 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2023"

# Read the tables
tables18 = pd.read_html(link18, header=0) #year 2018
tables19 = pd.read_html(link19, header=0) #year 2019
tables20 = pd.read_html(link20, header=0) #year 2020
tables21 = pd.read_html(link21, header=0) #year 2021
tables22 = pd.read_html(link22, header=0) #year 2022
tables23 = pd.read_html(link23, header=0) #year 2023

In [5]:
print(len(tables18),len(tables19),len(tables20),len(tables21),len(tables22),len(tables23))

13 13 13 13 13 13


In [6]:
def get_wiki_df(tables):
  if len(tables)>=6:
    # Perform the append operations
    df1, df2, df3, df4 = tables[2:6]
    df = pd.concat([df1, df2, df3, df4], ignore_index=True)
  else:
    # Handle the case where tables are not available
    df = pd.DataFrame() 
  return df
    

In [7]:
df18 = get_wiki_df(tables18)
df19 = get_wiki_df(tables19)
df20 = get_wiki_df(tables20)
df21 = get_wiki_df(tables21)
df22 = get_wiki_df(tables22)
df23 = get_wiki_df(tables23)

**Work on df1**

In [8]:
df1.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

We will work with actors and director name, genres and movie title.

In [9]:
df1 = df1.loc[:,['movie_title','genres','director_name','actor_1_name','actor_2_name','actor_3_name']]

genres are separated via '|', replace it with " "

In [10]:
df1['genres'] = df1['genres'].str.replace('|', ' ')

**Lets work on df2**

In [11]:
df2.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [12]:
df2 = df2.loc[:,['id','genres','title']]

id column in df2 contains some noise data.

In [13]:
is_present = '1997-08-20' in df2['id'].values 
print(is_present)

True


Remove rows where id format is not proper

In [14]:
# convert non-convertible id values to NaN
df2['id'] = pd.to_numeric(df2['id'], errors='coerce')

# Drop rows where 'id' is NaN (i.e., not convertible to int)
df2 = df2.dropna(subset=['id']) 

# convert id into int
df2['id'] = df2['id'].astype('int64')

**Work on df3**

In [15]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


**Combine df2 and df3**

In [16]:
merged_df = pd.merge(df2, df3, on='id', how='inner')

In [17]:
merged_df.head()

,id,genres,title,cast,crew
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Grumpier Old Men,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",Waiting to Exhale,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",Father of the Bride Part II,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [18]:
# convert string representations of lists or dictionaries back into actual Python objects 
import ast

def convert_to_objects(data):
    """
    Convert string representations of lists or dictionaries back into actual Python objects
    
    Args:
        data (str): The string representation of the data
        
    Returns:
        object: The converted Python object
    """
    return ast.literal_eval(data)

In [19]:
merged_df['genres'] = merged_df['genres'].map(convert_to_objects)
merged_df['cast'] = merged_df['cast'].map(convert_to_objects)
merged_df['crew'] = merged_df['crew'].map(convert_to_objects)

In [20]:
merged_df['genres_list'] = merged_df['genres'].apply(lambda x: ' '.join([genre.get('name', '') for genre in x]) or np.NaN)

In [21]:
# actors
merged_df['actor_1_name'] = merged_df['cast'].apply(lambda x: x[0]['name'] if x and len(x) > 0 else np.NaN)
merged_df['actor_2_name'] = merged_df['cast'].apply(lambda x: x[1]['name'] if x and len(x) > 1 else np.NaN)
merged_df['actor_3_name'] = merged_df['cast'].apply(lambda x: x[2]['name'] if x and len(x) > 2 else np.NaN)

# directors
merged_df['director_name'] = merged_df['crew'].apply(lambda x: next((crew['name'] for crew in x if crew.get('job') == 'Director'), np.NaN))

In [22]:
df4 = merged_df.loc[:,['title','genres_list', 'director_name','actor_1_name','actor_2_name','actor_3_name']]

In [23]:
df4 = df4.rename(columns={'title':'movie_title','genres_list':'genres'})

**Wikipedia Data processing**

Use of TMDB API for genres search

In [24]:
df18.head()

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],NaN
1,J A N U A R Y,5,The Strange Ones,Vertical Entertainment,Christopher Radcliff (director/screenplay); La...,[3],NaN
2,J A N U A R Y,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[4],NaN
3,J A N U A R Y,12,Proud Mary,Screen Gems,"Babak Najafi (director); John S. Newman, Chris...",[5],NaN
4,J A N U A R Y,12,Acts of Violence,Lionsgate Premiere,Brett Donowho (director); Nicolas Aaron Mezzan...,[6],NaN


In [25]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
import requests

tmdb = TMDb()
tmdb.api_key = '8080c6dff83cd358d743134fa5cb5282'
tmdb_movie = Movie()

In [26]:
def get_genre(movie):
    try:
        result = tmdb_movie.search(movie)
        if len(result) > 0:
            movie_id = result[0].id 
            response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}") 
            response.raise_for_status()  # Check for request errors
            json_data = response.json() 
            genres_list = json_data.get('genres', [])
            movie_genres = ' '.join(genre.get('name', '') for genre in genres_list) or np.NaN 
            return movie_genres
        else:
            return np.NaN
    except Exception as e:
        print(f"Error for movie '{movie}': {e}")
        return np.NaN

In [27]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director);")[0]
    elif " (directors)" in x:
        return x.split(" (directors);")[0]
    else:
        return x.split(" (director/screenplay);")[0]

In [28]:
def get_actor1(x):
    if len((x.split("(screenplay); ")[-1]).split(", "))>=1:
        return ((x.split("(screenplay); ")[-1]).split(", ")[0]) 
    else:
        return np.NaN

In [29]:
def get_actor2(x):
  if len((x.split("(screenplay); ")[-1]).split(", "))>=2:
    return ((x.split("(screenplay); ")[-1]).split(", ")[1]) 
  else:
    return np.NaN

In [30]:
def get_actor3(x):
  if len((x.split("(screenplay); ")[-1]).split(", "))>=3:
    return ((x.split("(screenplay); ")[-1]).split(", ")[2]) 
  else:
    return np.NaN

In [31]:
def get_wiki_process_data(df):
  df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))
  df['director_name'] = df['Cast and crew'].map(lambda x: get_director(str(x)))
  df['actor_1_name'] = df['Cast and crew'].map(lambda x: get_actor1(str(x)))
  df['actor_2_name'] = df['Cast and crew'].map(lambda x: get_actor2(str(x)))
  df['actor_3_name'] = df['Cast and crew'].map(lambda x: get_actor3(str(x)))
  df = df.loc[:,['Title','genres','director_name','actor_1_name','actor_2_name','actor_3_name']]
  df = df.rename(columns={'Title':'movie_title'})
  return df
  
  

**Process 2018 Movies**

In [32]:
len(df18)

241

In [33]:
df18 = get_wiki_process_data(df18) 

**Process 2019 Movies**

In [34]:
df19 = get_wiki_process_data(df19) 

**Process 2020 Movies**

In [35]:
df20 = get_wiki_process_data(df20) 

Error for movie 'No films had been released on this date as resulting the closure of theaters caused by the COVID-19 pandemic': getattr(): attribute name must be string
Error for movie 'The SpongeBob Movie: Sponge on the Run (Canadian theatrical release)': getattr(): attribute name must be string


**Process 2021 Movies**

In [36]:
df21 = get_wiki_process_data(df21) 

Error for movie 'Dating and New York': getattr(): attribute name must be string


**Process 2022 Movies**

In [37]:
df22 = get_wiki_process_data(df22) 

Error for movie 'Apollo 10 1⁄2: A Space Age Childhood': getattr(): attribute name must be string


**Process 2023 Movies**

In [38]:
df23 = get_wiki_process_data(df23) 

**Merge and Clean**

We get first level clean data as df1, df4 (got from df2 and df3), df18 to df23. 
Lets merge it to a single df and preprocess that.

In [39]:
df1.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Avatar,Action Adventure Fantasy Sci-Fi,James Cameron,CCH Pounder,Joel David Moore,Wes Studi
1,Pirates of the Caribbean: At World's End,Action Adventure Fantasy,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport
2,Spectre,Action Adventure Thriller,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman
3,The Dark Knight Rises,Action Thriller,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt
4,Star Wars: Episode VII - The Force Awakens ...,Documentary,Doug Walker,Doug Walker,Rob Walker,NaN


In [40]:
df4.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Toy Story,Animation Comedy Family,John Lasseter,Tom Hanks,Tim Allen,Don Rickles
1,Jumanji,Adventure Fantasy Family,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst
2,Grumpier Old Men,Romance Comedy,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret
3,Waiting to Exhale,Comedy Drama Romance,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine
4,Father of the Bride Part II,Comedy,Charles Shyer,Steve Martin,Diane Keaton,Martin Short


In [41]:
df18.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Horror Mystery Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Drama Mystery,Christopher Radcliff (director/screenplay); La...,Christopher Radcliff (director/screenplay); La...,James Freedson-Jackson,Emily Althaus
2,The Commuter,Action Thriller Mystery,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson
3,Proud Mary,Thriller Action Crime,Babak Najafi,Taraji P. Henson,Jahi Di'Allo Winston,Billy Brown
4,Acts of Violence,Action Crime Thriller,Brett Donowho,Bruce Willis,Cole Hauser,Shawn Ashmore


In [42]:
df19.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Escape Room,Horror Thriller Mystery,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll
1,Rust Creek,Thriller Drama Action Crime,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan
2,American Hangman,Thriller,Wilson Coneybeare,Wilson Coneybeare (director/screenplay); Donal...,Vincent Kartheiser,Oliver Dennis
3,A Dog's Way Home,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp
4,The Upside,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman


In [43]:
df20.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Horror Mystery,Nicolas Pesce,Nicolas Pesce (director/screenplay); Andrea Ri...,Demián Bichir,John Cho
1,Underwater,Horror Science Fiction Action Adventure,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs


In [44]:
df21.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,Hacksaw,Horror,Anthony Leone,Anthony Leone (director/screenplay); Amy Cay,Brian Patrick Butler,Michael C. Burgess
2,Dr. Bird's Advice for Sad Poets,Comedy Drama,Yaniv Raz,Yaniv Raz (director/screenplay); Lucas Jade Zu...,Taylor Russell,Chase Stokes
3,The White Tiger,Drama,Ramin Bahrani,Ramin Bahrani (director/screenplay); Adarsh Go...,Rajkummar Rao,Priyanka Chopra Jonas
4,Locked Down,Comedy Crime Romance,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant


In [45]:
df22.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The 355,Action Thriller,Simon Kinberg,Jessica Chastain,Penélope Cruz,Fan Bingbing
1,The Legend of La Llorona,Family Animation Fantasy Horror,Patricia Harris Seeley,Autumn Reeser,Danny Trejo,Antonio Cupo
2,The Commando,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,American Siege,Action Adventure Thriller,Edward John Drake,Edward John Drake (director/screenplay); Timot...,Bruce Willis,Rob Gough
4,Scream,Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega


In [46]:
df23.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,M3GAN,Science Fiction Horror,Gerard Johnstone,Allison Williams,Violet McGraw,Amie Donald
1,The Old Way,Western,Brett Donowho,Nicolas Cage,Ryan Kiera Armstrong,NaN
2,The Devil Conspiracy,Horror Fantasy Science Fiction Thriller,Nathan Frankowski,Alice Orr-Ewing,Joe Doyle,Eveline Hall
3,Plane,Action,Jean-François Richet,Gerard Butler,Mike Colter,Yoson An
4,House Party,Comedy,Calmatic,Tosin Cole,Jacob Latimore,Karen Obilom


Second level of Preprocessing

In [47]:
# Concatenate DataFrames vertically
df = pd.concat([df1,df4,df18,df19,df20,df21,df22,df23], ignore_index=True)

df['movie_title'] = df['movie_title'].str.lower()

# Remove duplicate rows
df = df.drop_duplicates(subset='movie_title')

# Resetting the index after dropping duplicates
df.reset_index(drop=True, inplace=True)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48601 entries, 0 to 48600
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   movie_title    48600 non-null  object
 1   genres         46321 non-null  object
 2   director_name  47638 non-null  object
 3   actor_1_name   46294 non-null  object
 4   actor_2_name   44984 non-null  object
 5   actor_3_name   44033 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


Fill missing values as Unknown.

In [49]:
df['movie_title'].fillna('unknown',inplace=True)
df['genres'].fillna('unknown',inplace=True)
df['director_name'].fillna('unknown',inplace=True)
df['actor_1_name'].fillna('unknown',inplace=True)
df['actor_2_name'].fillna('unknown',inplace=True)
df['actor_3_name'].fillna('unknown',inplace=True)


New Feature: combine some features together to hold more context. 

In [50]:
df['all_info'] = df['actor_1_name'] + ' ' + df['actor_2_name'] + ' '+ df['actor_3_name'] + ' '+ df['director_name'] +' ' + df['genres']

Check for null entries

In [51]:
df.isna().sum()

movie_title      0
genres           0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
all_info         0
dtype: int64

In [52]:
len(df)

48601

In [53]:
df.to_csv('../data/data.csv',index=False)